<a href="https://colab.research.google.com/github/shiffa-04/Titanic-ML-Model/blob/main/T_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers
import joblib

In [3]:
df = pd.read_csv("modified_file.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [7]:
columns_to_drop = ['Survived','PassengerId', 'Ticket', 'Fare', 'Name']
X = df.drop(columns = columns_to_drop)
y= df['Survived']

print(X.head())


   Pclass     Sex   Age  SibSp  Parch Embarked
0       3    male  22.0      1      0        S
1       1  female  38.0      1      0        C
2       3  female  26.0      0      0        S
3       1  female  35.0      1      0        S
4       3    male  35.0      0      0        S


In [8]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
   transformers = [('num',numerical_transformer,['Age','SibSp', 'Parch'] ),
                   ('cat', categorical_transformer,['Pclass', 'Sex','Embarked'])
   ])

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state = 10)

In [9]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

X_train_shape = X_train.shape
X_test_shape = X_test.shape

print(f'Shape of X_train :{X_train_shape}')
print(f'Shape of X_test:{X_test_shape}')

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



Shape of X_train :(712, 12)
Shape of X_test:(179, 12)


In [10]:
model = Sequential(
    [
        Dense(units = 64, input_dim = X_train.shape[1], kernel_regularizer=regularizers.l2(0.01), activation = 'relu', name='l1'),
        Dropout(rate=0.3, name='dropout1'),
        Dense(units = 1, activation = 'sigmoid', name='l3')
    ]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [13]:
# batch_size = 20
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4205 - loss: 0.9321 - val_accuracy: 0.7654 - val_loss: 0.8193
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6714 - loss: 0.8090 - val_accuracy: 0.8101 - val_loss: 0.7367
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7575 - loss: 0.7391 - val_accuracy: 0.8324 - val_loss: 0.6736
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7702 - loss: 0.6963 - val_accuracy: 0.8380 - val_loss: 0.6253
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8101 - loss: 0.6347 - val_accuracy: 0.8268 - val_loss: 0.5924
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7906 - loss: 0.6238 - val_accuracy: 0.8212 - val_loss: 0.5683
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8080 - loss: 0.5884 - val_accuracy: 0.8268 - val_loss: 0.5440
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8215 - loss: 0.5484 - val_accuracy: 0.8212 - 

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100 :.2f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8698 - loss: 0.3800 
Test Accuracy: 84.92


In [15]:
joblib.dump(model, 'model.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']